# Excel Data Extractor 

This function extracts all column headers for all visible sheets from a list of excel files.

Requirements: Run function in folder containing the source folder.

Usage: ExtractExcelData(source_folder=None)

Output: Creates a folder (named "Extracted_Fields") of excel files with each file listing sheetnames (Column A) and associated column headers (Column B)

Return: (source_folder, merged_df_dict): tuple of Source folder name (string) and dictionary of merged dataframes for each source excel file

        

In [ ]:
def ExtractExcelData(source_folder):
                     
    import os
    import pandas as pd
    import re
    import pickle
    
    if source_folder==None:
        print('Missing source_folder input!')
        return None    
    
    cwd=os.getcwd()
    # List all files to extract
    files=[]
    for file in os.listdir(os.path.join(cwd,source_folder)):
        if '.xlsx'in file:
            files.append(file)
    #print(files)
    
    # Access source folder
    regex=re.compile(r'.*\\{0}$'.format(source_folder))
    if bool(regex.match(os.getcwd()))==False:
        os.chdir('.\\{0}'.format(source_folder))
    
    merged_df_dict=dict()
    # Loop through all .xlsx files
    for i, file in enumerate(files):
        filename=file
        
        try:
            a=0
            # Retrieve sheets from file and store sheetnames in list
            sourcefile_name=filename
            a=1
            wb=pd.ExcelFile(sourcefile_name)
            sourcefile_shts=wb.book.sheets()
            a=2
            # Extract information from all sheets. Header is a multiindex
            sourcefile_dfs=[]
            print('\nExtracting File {0}/{1} - {2}'.format(i+1,len(files),filename))
            for sht in sourcefile_shts:
                if sht.visibility == 1 or sht.visibility == 2:
                    continue            
                print('Extracting {}'.format(sht.name))
                sourcefile_df=pd.read_excel(wb,sheet_name=sht.name,header=0, keep_default_na=False)
                sourcefile_df=sourcefile_df.groupby(sourcefile_df.columns[0]).agg(lambda x: ', '.join(set(x.astype(str))))
                sourcefile_df_cols=sourcefile_df.columns.tolist()
                multi_idx=pd.MultiIndex.from_product([[sht.name],sourcefile_df_cols])
                sourcefile_df.columns=multi_idx
                sourcefile_dfs.append(sourcefile_df) 
            a=3
            # Merge all sheets into single df
            #print('Merging sheet {0}/{1}'.format(2,len(sourcefile_dfs)))
            merged_df=sourcefile_dfs[0].merge(sourcefile_dfs[1],left_index=True,right_index=True,how='outer')
            for i in range(2,len(sourcefile_dfs)):
                #print('Merging sheet {0}/{1}'.format(i+1,len(sourcefile_dfs))) 
                merged_df=merged_df.merge(sourcefile_dfs[i],left_index=True,right_index=True,how='outer')

            # Rename index
            merged_df.index.rename('SPEC ID', inplace=True)
            merged_df_dict[sourcefile_name]=merged_df
            
            a=4
            # Create df to store headers
            fields=merged_df.columns.tolist()
            fields_df=pd.DataFrame(fields,columns=['Sheet Name','Column Name'])
            fields_df['Template Column Name']=""

            # Create folder to store excel files
            regex3=re.compile(r'.*\\Extracted_Fields$')
            #If not in Extracted_Fields folder already
            if bool(regex3.match(os.getcwd()))==False:
                if 'Extracted_Fields' not in os.listdir():
                        os.mkdir('.\\Extracted_Fields')
                        os.chdir('.\\Extracted_Fields')
                else:    
                    os.chdir('.\\Extracted_Fields')

            # Write extracted fields into destination folder
            writer = pd.ExcelWriter("{0}_Extracted_Fields.xlsx".format(os.path.splitext(filename)[-2]), engine='xlsxwriter')
            sheetname='Extracted Fields'
            fields_df.to_excel(writer, sheet_name=sheetname,index=False)  # send df to writer
            worksheet = writer.sheets[sheetname]  # pull worksheet object

            for idx, col in enumerate(fields_df):  # loop through all columns
                    series = fields_df[col]
                    max_len = max((
                        series.astype(str).map(len).max(),  # len of largest item
                        len(str(series.name))  # len of column name/header
                        )) + 1  # adding a little extra space
                    worksheet.set_column(idx, idx, max_len)  # set column width .set_column(first_col, last_col, width, cell_format, options)
            writer.save()
            os.chdir(os.path.join(cwd,source_folder))
            print("File created! - {0}_Extracted_Fields.xlsx".format(os.path.splitext(filename)[-2]))

        except:
            #files.clear()
            os.chdir(cwd)
            print('Error detected while extracting file {0} - {1}\nFailed at Step {2}'.format(i+1,file,a))
            return None
      
    #Back to starting point
    os.chdir(cwd)
    
    #Write merged_df_dict into pickle file
    with open('ExtractExcelDataObjects.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([source_folder,merged_df_dict], f)
    
    print("\nFields extracted!")
    return (source_folder, merged_df_dict)


In [ ]:
source_folder, merged_df_dict=ExtractExcelData(source_folder='Source Folder')